# translate

In [11]:
from googletrans import Translator
import langid
import pandas as pd

In [24]:
def detect_Translator(text):
    translator = Translator()
    try:
        detection = translator.detect(text)
        return detection
    except Exception as e:
        return "error:- "+e

In [25]:
def detect_langid(text):
    try:
        lang, confidence = langid.classify("Sample Hinglish text here")
        return lang
    except Exception as e:
        return "error:- "+e

In [29]:
def test(df : pd.DataFrame):
    for index,row in df.iterrows():
        langid = detect_langid(row['Comment'])
        Translator = detect_Translator(row['Comment'])
        print(f"{row['Comment']}, langid :- {langid}, Translator :- {Translator}")
        print('\n')

In [ ]:
df=pd.read_csv(r'C:\Users\tarun\Desktop\Youtube-Comments-Sentiments-Analysis\comments_df.csv')
test(df)

In [47]:
def detect_and_translate(text):
    translator = Translator()
    try:
        detection = translator.detect(text)
        print(detection)
        if detection.lang != 'en':
            translation = translator.translate(text, dest='en')
            if translation and translation.text:
                print(translation.text)
                print(translation)
                return translation.text
            else:
                return 'This is a neutral text'
        else:
            return text
    except Exception as e:
        print(f"Translation error: {e}")
        return 'This is a neutral text'

In [65]:
ans=detect_and_translate(text)
ans

Detected(lang=hi, confidence=None)
A desert will not die now, Betiichod
Translated(src=hi, dest=en, text=A desert will not die now, Betiichod, pronunciation=None, extra_data="{'confiden...")


'A desert will not die now, Betiichod'

# Text preprocessing

In [2]:
import emoji
import re

def replace_emojis_with_meanings(text):
    # First, demojize the text to get :emoji_name: format
    demojized_text = emoji.demojize(text)
    
    # Use regex to find all :emoji_name: patterns and replace them with readable text
    def convert_match(match):
        # Get the matched emoji name without colons
        emoji_name = match.group(1)
        # Replace underscores with spaces
        readable_name = emoji_name.replace('_', ' ')
        return readable_name
    
    # Replace all :emoji_name: with readable text
    readable_text = re.sub(r':([a-zA-Z0-9_]+):', convert_match, demojized_text)
    
    return readable_text

In [5]:
def remove_emojis(text):
  return emoji.replace_emoji(text, replace="")

In [12]:
df=pd.read_csv(r"C:\Users\tarun\Desktop\Youtube-Comments-Sentiments-Analysis\comments_df.csv")

In [17]:
df.shape[0]

20

# sentiment comments sagregation 

In [3]:
import os
import re
import emoji
import awsgi
import string
import numpy as np
import pandas as pd
import tensorflow as tf
from textblob import TextBlob
from dotenv import load_dotenv
from googletrans import Translator
from googleapiclient.discovery import build
from transformers import AutoTokenizer
from flask import Flask, render_template, request

c:\Users\tarun\Desktop\Youtube-Comments-Sentiments-Analysis\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df=pd.read_csv(r"C:\Users\tarun\Desktop\Youtube-Comments-Sentiments-Analysis\final_df.csv")

In [6]:
sentiment_counts = {'Sadness': 0, 'Joy': 0, 'Love': 0, 'Annoyed': 0, 'Fear': 0, 'Surprise': 0}
comments_by_sentiment = {'Sadness': [], 'Joy': [], 'Love': [], 'Annoyed': [], 'Fear': [], 'Surprise': []}
SENTIMENT_LABELS = {0: 'Sadness', 1: 'Joy', 2: 'Love', 3: 'Annoyed', 4: 'Fear', 5: 'Surprise'}

In [9]:
tokenizer_path = r'C:\Users\tarun\Desktop\Youtube-Comments-Sentiments-Analysis\model\saved_tokenizer'
model_path = r'C:\Users\tarun\Desktop\Youtube-Comments-Sentiments-Analysis\model\transformer'
fine_tuned_tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
fine_tuned_model = tf.saved_model.load(model_path)

In [7]:
def return_sentiment(text : str) -> np.int64 :
    inputs = fine_tuned_tokenizer(text, return_tensors="tf", padding="max_length", truncation=True, max_length=55)
    input_dict = {
        'input_ids': tf.cast(inputs['input_ids'], tf.int64),
        'attention_mask': tf.cast(inputs['attention_mask'], tf.int64),
        'token_type_ids': tf.cast(inputs['token_type_ids'], tf.int64)
        }
    outputs = fine_tuned_model(input_dict)
    probabilities = tf.nn.softmax(outputs, axis=-1) 
    probabilities_np = probabilities.numpy()[0] 
    predicted_index = np.argmax(probabilities_np)
    return predicted_index

In [ ]:
for index, row in df.iterrows():
        sentiment_index = return_sentiment(row['Comment'])  
        sentiment_label = SENTIMENT_LABELS[sentiment_index] 
        sentiment_counts[sentiment_label]+=1
        comments_by_sentiment[sentiment_label].append(translated_comment)

In [ ]:
sentiment_counts

In [ ]:
comments_by_sentiment